In [10]:
import sympy as sp
import numpy as np

CV = 0.25
l_x = 8
l_y = 16
l_z = 110
delta_block = 1

dx = sp.symbols('dx')

Xblock = 2*delta_block + l_x*3 + 2*dx
Yblock = 2*delta_block + l_y*3 + 2*dx
Zblock = 2*delta_block + l_z 
rho_ecv = 1 - (9*l_x*l_y*l_z)/(Xblock*Yblock*Zblock)

# Print the expanded equation
equation = rho_ecv - CV
print("Expanded equation:")
print(equation, "= 0")

# Find real solutions only
real_solutions = sp.solve(equation, dx, domain='RR')

print("\nReal solutions:")
print(real_solutions)

# If solutions exist, validate them
if real_solutions:
    for sol in real_solutions:
        # Use N() for numerical evaluation instead of float conversion
        sol_num = np.complex128(sol)
        sol_num = sol_num.real
        sol_num = float(sol_num)
        print(type(sol_num))
        print(f"\nValidation for dx = {sol_num}:")
        # Check if this solution gives the desired CV
        result = 1 - (9*l_x*l_y*l_z)/((2*delta_block + l_x*3 + 2*sol_num)*(2*delta_block + l_y*3 + 2*sol_num)*(2*delta_block + l_z))
        print(f"Resulting CV = {sp.N(result):.6f}")

Expanded equation:
0.75 - 7920/(7*(2*dx + 26)*(2*dx + 50)) = 0

Real solutions:
[-39.3259158992370, 1.32591589923704]
<class 'float'>

Validation for dx = -39.32591589923704:
Resulting CV = 0.250000
<class 'float'>

Validation for dx = 1.3259158992370415:
Resulting CV = 0.250000


In [14]:
dx = 1.33
rho_ecv = 1-(9*l_x*l_y*l_z)/((2*delta_block + l_x*3 + 2*dx)*(2*delta_block + l_y*3 + 2*dx)*(2*delta_block + l_z))
print(f"ECV = {rho_ecv*100:.2f}%")

ECV = 25.03%


Function to remove lines to .off file (postprocess)

In [5]:
def remove_lines_from_file(file_path, line_numbers_to_remove):
    """
    Remove specific lines from a file.
    
    :param file_path: Path to the file.
    :param line_numbers_to_remove: List of line numbers to remove (1-based indexing).
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    with open(file_path, 'w') as file:
        for i, line in enumerate(lines, start=1):
            if i not in line_numbers_to_remove:
                file.write(line)

Function to add 3D rectangular prism given the sides lx,ly,lz and center (on low=z) through gmsh py

In [6]:
import gmsh

def add_cuboid(center, lx, ly, lz, lc=10.):
    cx, cy, cz = center
    hx, hy = lx / 2, ly / 2
    
    # Define the points of the cuboid:
    p1 = gmsh.model.geo.add_point(cx - hx, cy - hy, cz, lc)
    p2 = gmsh.model.geo.add_point(cx + hx, cy - hy, cz, lc)
    p3 = gmsh.model.geo.add_point(cx + hx, cy + hy, cz, lc)
    p4 = gmsh.model.geo.add_point(cx - hx, cy + hy, cz, lc)
    
    p5 = gmsh.model.geo.add_point(cx - hx, cy + hy, cz + lz, lc)
    p6 = gmsh.model.geo.add_point(cx - hx, cy - hy, cz + lz, lc)
    p7 = gmsh.model.geo.add_point(cx + hx, cy - hy, cz + lz, lc)
    p8 = gmsh.model.geo.add_point(cx + hx, cy + hy, cz + lz, lc)

    # Define the edges of the cuboid:
    l1 = gmsh.model.geo.add_line(p1, p2)
    l2 = gmsh.model.geo.add_line(p2, p3)
    l3 = gmsh.model.geo.add_line(p3, p4)
    l4 = gmsh.model.geo.add_line(p4, p1)
    l5 = gmsh.model.geo.add_line(p5, p6)
    l6 = gmsh.model.geo.add_line(p6, p7)
    l7 = gmsh.model.geo.add_line(p7, p8)
    l8 = gmsh.model.geo.add_line(p8, p5)
    
    l9 = gmsh.model.geo.add_line(p4, p5)
    l10 = gmsh.model.geo.add_line(p6, p1)
    l11 = gmsh.model.geo.add_line(p7, p2)
    l12 = gmsh.model.geo.add_line(p3, p8)

    # Define the faces of the cuboid:
    cl1 = gmsh.model.geo.add_curve_loop([-l1, -l2, -l3, -l4])
    cl2 = gmsh.model.geo.add_curve_loop([l5, l6, l7, l8])
    cl3 = gmsh.model.geo.add_curve_loop([-l9, -l5, -l10, +l4])
    cl4 = gmsh.model.geo.add_curve_loop([l9, -l8, -l12, l3])
    cl5 = gmsh.model.geo.add_curve_loop([-l6, -l11, +l1, +l10])
    cl6 = gmsh.model.geo.add_curve_loop([l11, l2, l12, -l7])

    # Create the surfaces of the cuboid:
    s1 = gmsh.model.geo.add_plane_surface([cl1])
    s2 = gmsh.model.geo.add_plane_surface([cl2])
    s3 = gmsh.model.geo.add_plane_surface([cl3])
    s4 = gmsh.model.geo.add_plane_surface([cl4])
    s5 = gmsh.model.geo.add_plane_surface([cl5])
    s6 = gmsh.model.geo.add_plane_surface([cl6])

    # Synchronize to create the volume:
    gmsh.model.geo.synchronize()

    # Create a surface loop and volume:
    sl = gmsh.model.geo.add_surface_loop([s1, s2, s3, s4, s5, s6])

In [17]:
import trimesh

center_x = delta_block + l_x/2
center_y = delta_block + l_y/2
center_z = delta_block
counter = 0

for i in range (0,3):
    for j in range (0,3):
        center = [center_x, center_y]
        center[0] += (l_x+delta_block)*i
        center[1] += (l_y+delta_block)*j
        gmsh.initialize()
        
        add_cuboid((center[0], center[1], center_z), l_x, l_y, l_z)

        gmsh.model.mesh.generate(2)

        # Write mesh data to file:
        gmsh.write("simple_domain/cuboid_{0}.msh".format(counter))

        gmsh.finalize()

        import meshio

        mesh = meshio.read("simple_domain/cuboid_{0}.msh".format(counter))

        triangle_cells = [cell for cell in mesh.cells if cell.type == "triangle"]

        triangle_mesh = meshio.Mesh(points=mesh.points, cells=triangle_cells)

        triangle_mesh.write(
            "simple_domain/cuboid_{0}.ply".format(counter),  # str, os.PathLike, or buffer/open file
            file_format="ply"
        )
        
        mesh_check = trimesh.load("simple_domain/cuboid_{0}.ply".format(counter))
        
        if not mesh_check.is_watertight:
            print("Mesh is not watertight.")
            raise RuntimeError("Mesh is not watertight.")
        
        # Ensure normals are consistent and pointing outward
        if not mesh_check.is_winding_consistent:
            mesh_check.fix_normals()
                
        # Make sure volume is positive
        if mesh_check.volume < 0:
            mesh_check.invert()
                
        mesh_check.remove_degenerate_faces()
        mesh_check.remove_duplicate_faces()
        
        if not mesh_check.is_watertight:
            print("Mesh is not watertight after fixing.")
            raise RuntimeError("Mesh is not watertight after fixing.")
        
        if not mesh_check.is_winding_consistent:
            print("Mesh winding is not consistent after fixing.")
            raise RuntimeError("Mesh winding is not consistent after fixing.")

        mesh_check.export("simple_domain/cuboid_{0}.off".format(counter))
        print(f"Mesh successfully exported to simple_domain/cuboid_{0}.off")
                
        # triangle_mesh.write(
        #     "simple_domain/cuboid_{0}.off".format(counter),  # str, os.PathLike, or buffer/open file
        #     file_format="off"
        # )
        # remove_lines_from_file("simple_domain/cuboid_{0}.off".format(counter), [2,3,5])
        
        counter += 1

Info    : Increasing process stack size (8176 kB < 16 MB)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.000623458s, CPU 0.000696s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 70%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 6 (Plane, Fr